In [1]:
from cs231n.data_utils import get_CIFAR10_data

# load cifar10 data
data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [2]:
# split and normalize data
X_val, y_val = data['X_val'], data['y_val']
X_train, y_train = data['X_train'], data['y_train']
X_test, y_test = data['X_test'], data['y_test']

# X_train /= 255.
# X_val /= 255.
# X_test /= 255.

In [3]:
import keras
from keras.utils import to_categorical

# 1 hot encode labels
y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)
y_test = to_categorical(y_val, num_classes=10)

Using TensorFlow backend.


In [4]:
# Conv -> Batch -> Relu -> Conv -> Batch -> Relu -> Pool -> x 2
# Conv -> Batch -> Relu -> Conv -> Batch -> Relu -> Conv -> Batch -> Relu -> Pool -> x2
# Affine - > Affine -> Softmax

# Windows were all 3x3, stride 1.
# Pooling were all 2x2 max.

# Total conv layers 10 with number of filters - 32, 32, 64, 64, 128, 128, 128, 196, 196, 196.
# Affine layers had 500 hidden dims.

# conv-relu-conv-relu-pool x 2
# affine-softmax
# lrf's are all 3x3, stride 1
# pooling are all 2x2 max

In [19]:
# build network
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential

import numpy as np

model = Sequential()

def my_init(shape, dtype=None):
    weight_scale = 1e-3
    return weight_scale * np.random.randn(*shape).astype(dtype)

keras.backend.set_image_data_format('channels_first')
conv_kwargs = {'filters': 32, 'kernel_size': 3, 
               'padding': 'same', 'kernel_initializer': my_init}
batch_kwargs = {'momentum': 0.9,
                'epsilon': 1e-5, 'moving_variance_initializer': 'zeros'}

shape = X_train.shape[1:]
model.add(Conv2D(input_shape=shape, **conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(Conv2D(**conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(MaxPool2D())

model.add(Conv2D(**conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(Conv2D(**conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(500, kernel_initializer=my_init))
model.add(BatchNormalization(**batch_kwargs))
model.add(Activation('relu'))
model.add(Dense(10, kernel_initializer=my_init))
model.add(Activation('softmax'))

In [20]:
# set optimizer
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
# train model
history = model.fit(X_train, y_train,
                    batch_size=100, epochs=1,
                    validation_data=(X_val, y_val),
                    shuffle=True)

Train on 49000 samples, validate on 1000 samples
Epoch 1/1
49000/49000 [==============================] - 2221s - loss: 1.2195 - acc: 0.5600 - val_loss: 0.9217 - val_acc: 0.6720
